Aggregating data to the level of individuals

In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from matplotlib import pyplot as plt
import pandas as pd
from bsm import individuals
from bsm import readVCF
import statsmodels as sm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
calls, clin = individuals.get_data()

Welcome, Attila Jones!



CHROM       3301
POS         3301
REF         3301
ALT         3301
QUAL        3301
            ... 
EV_2        3154
EV_3        3154
EV_4        3154
EV_5        3154
Mutation    3301
Length: 126, dtype: int64

In [5]:
%connect_info

{
  "shell_port": 59457,
  "iopub_port": 44589,
  "stdin_port": 58501,
  "control_port": 46721,
  "hb_port": 49643,
  "ip": "127.0.0.1",
  "key": "1c5afd4c-0a2747ed6e8a0ab06497ccd0",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-d0abaac7-ba87-4b77-84f0-1c17c456bb83.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
